In [ ]:
# Descargar docker: https://www.docker.com/products/docker-desktop/
# En terminal para descargar neo4j: docker pull neo4j
# Empezar neo4j: docker run --name neo4j-container -p 7474:7474 -p 7687:7687 -e NEO4J_AUTH=neo4j/password -d neo4j
# Usar neo4j: http://localhost:7474/browser/
# Para ver el grafo: MATCH (n)-[r]->(m) RETURN n, r, m
# Para eliminarlo: MATCH (n) DETACH DELETE n

In [12]:
!pip install neo4j


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from neo4j import GraphDatabase
import csv

# Configuración de conexión
uri = "bolt://localhost:7687"
user = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(user, password))

# Ruta al CSV
csv_file = 'simbolico_spacy_juan_rana.csv'

# Abrir sesión
session = driver.session()

# Leer CSV y cargar relaciones
with open(csv_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        head = row['Head']
        relation = row['Relation'].strip().lower().replace(" ", "_").replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")
        tail = row['Tail']

        # Cypher: crear nodos y relación
        cypher_query = f"""
        MERGE (h:Entity {{name: $head}})
        MERGE (t:Entity {{name: $tail}})
        MERGE (h)-[r:{relation}]->(t)
        """

        # Ejecutar query
        session.run(cypher_query, head=head, tail=tail)

# Cerrar sesión
session.close()
driver.close()
